# Домашнее задание 1

##### Настя Скутина (184)

Сначала читаем файл:

In [175]:
import json

twitter = []
for line in open('hw_3_twitter.json'):
    twitter.append(json.loads(line))

**Вопрос 1.** Сколько твитов в наборе?

Каждый элемент списка - 1 твит, значит нужно найти длину списка:

In [176]:
print('В наборе', len(twitter), 'твитов.')

В наборе 2556 твитов.


**Вопрос 2.** Какой процент твитов составляют удаленные записи?

Составляем список удаленных твитов, находим его длину и делим на длину списка twitter:

In [177]:
deleted = []
for tweet in twitter:
    if 'delete' in tweet:
        deleted.append(tweet)

percent = str(int(len(deleted) / len(twitter) * 100))
print('Удаленные записи составляют', percent+'%', 'от всего набора.')

Удаленные записи составляют 14% от всего набора.


**Вопрос 3.** Какие самые популярные языки твитов?

Сначала нужно найти существующие (неудаленные) твиты.
Затем находим язык каждого твита, после чего применяем Counter и получаем ранжированный по количеству языков список

In [178]:
from collections import Counter

tweets = []
for tweet in twitter:
    if 'delete' not in tweet:
        tweets.append(tweet)

languages = []
for tweet in tweets:
    languages.append(tweet['lang'])

lang = Counter(languages)
place = 0
print('Топ-10 самых популярных языков:\n')
for elem in lang.most_common(10):
    place += 1
    print(str(place)+'. ', elem[0])

Топ-10 самых популярных языков:

1.  en
2.  ja
3.  es
4.  ko
5.  th
6.  ar
7.  und
8.  in
9.  pt
10.  fr


**Вопрос 4.** Есть ли твиты от одного и того же пользователя? Если да, то сколько таких пользователей?

Создаем два списка: для id всех пользователей и только для повторяющихся. Проходим по каждому твиту и если его автора еще нет в списке всех пользователей, то добавляем его id туда, а если есть, то уже в список повторяющихся (так как он встречается уже не в первый раз). В конце просто выводим длину этого списка.

In [179]:
users = []
repeating_users = []
for tweet in tweets:
    if tweet['user']['id'] in users:
        repeating_users.append(tweet['user']['id'])
    else:
        users.append(tweet['user']['id'])

print('В наборе', len(repeating_users), 'повторяющихся пользователей.')

В наборе 25 повторяющихся пользователей.


**Вопрос 5.** Топ-20 хэштегов

Массив с хэштегами находится в массиве entities. Поэтому нужно просто записать в список все хэштеги, а потом применить к нему Counter, чтобы получить ранжированный список.

In [181]:
hashtags = []
for tweet in tweets:
    for hashtag in tweet['entities']['hashtags']:
        hashtags.append(hashtag['text'])

print('Топ-20 Хэштегов:\n')
place = 0
for tag in Counter(hashtags).most_common(20):
    print('{0:<40} \t {1:<3}'.format(tag[0], tag[1]))

Топ-20 Хэштегов:

BTS                                      	 17 
방탄소년단                                    	 13 
AMAs                                     	 11 
人気投票ガチャ                                  	 8  
태형                                       	 7  
뷔                                        	 6  
BTSinChicago                             	 5  
BTSLoveYourselfTour                      	 5  
오늘의방탄                                    	 5  
PledgeForSwachhBharat                    	 5  
MPN                                      	 5  
PCAs                                     	 4  
V                                        	 4  
시카고1회차공연                                 	 4  
เป๊กผลิตโชค                              	 4  
JIMIN                                    	 4  
running                                  	 3  
NCT                                      	 3  
지민                                       	 3  
WajahmuPlastik                           	 3  


**Вопрос 6.** Предобработать тексты оригинальных твитов (не ретвитов) на английском языке (убрать пунктуацию, привести к нижнему регистру) и составить частотный словарь.

Для этого мы складываем все тексты, удовлетворяющие требованиям ('lang' = 'en', полей 'retweeted_status' и 'quoted_status' нет) в отдельный список, после чего с помощью регулярных выражений очищаем текст от ссылок (ссылки на изображения очень часто встречаются и могут повлиять на ответ) и создаем список слов, очищенных от пробелов и знаков препинания. Потом проходим по списку с помощью Counter и сортируем по частоте.

In [187]:
import re

en_original_texts = []
for tweet in tweets:
    if tweet['lang'] == 'en':
        if 'retweeted_status' not in tweet and 'quoted_status' not in tweet:
            en_original_texts.append(tweet['text'])

en_words = []
for text in en_original_texts:
    text = text.lower()
    new_text = re.sub('https://t.co/.*?$', '', text)
    words = re.findall('[\\d\\w]+', new_text)
    for word in words:
        en_words.append(word)

print('Частотный словарь (первые 20 слов)\n')
for word in Counter(en_words).most_common(20):
    print(word[0], word[1], sep='\t')

Частотный словарь (первые 20 слов)

the	106
i	80
to	78
a	69
and	58
it	51
you	50
is	41
for	40
of	40
that	37
s	37
t	33
in	32
me	26
on	25
my	25
be	22
are	20
this	19


**Вопрос 7.** Найти количество подписчиков (фолловеров) у авторов твитов и вывести топ-10

Составляем словарь, в котором ключ - имя пользователя, а значение - его количество подписчиков. Применяем к нему Counter и выводим с помощью format() красивую таблицу.

In [188]:
user_followers = {}
for tweet in tweets:
    user_followers[tweet['user']['name']] = tweet['user']['followers_count']

print('Топ-10 популярных авторов\n')
for item in Counter(user_followers).most_common(10):
    print('{0:<25} \t {1:<8}'.format(item[0], item[1]))

Топ-10 популярных авторов

Filosofía♕                	 2521403 
FITNESS Magazine          	 1491309 
malaysiakini.com          	 1206759 
NYT Science               	 1137374 
Gramática                 	 625463  
TGRT Haber                	 392472  
The Sun Football ⚽        	 383698  
Melbourne, Australia      	 374222  
Roznama Express           	 318189  
💞 ცųཞɠɛཞცơơɠıɛ 💞          	 311319  
